[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rycroft-group/math714/blob/main/c_bvp/bvp.ipynb)

In [ ]:
# Necessity libraries
import numpy as np
import matplotlib.pyplot as plt
from math import *
from scipy.optimize import fsolve

# Optional: a library for plotting with LaTeX-like 
# styles nicer formatted figures
# Warning: need to have LaTeX installed
import scienceplots
plt.style.use(['science'])

# Boundary value problem

## Method of manufactured solutions

Suppose we wish to measure numerical error of our equation
$$
u'' = f
$$
where $u(0) = \alpha$ and $u(1) = \beta$.

One approach is to write down an exact analytical solution, such as when $f(x) = 1$ then
$$
u(x) = \frac{x(x-1)}{2}
$$
with $u(0) = u(1) = 0$. Then we can compare our numerical solution to this analytical one.

_But for more complicated equations, it may not be possible to construct an analytical solution._

In this case, we can use the __method of manufactured solutions__:
- Choose a solution $u$ that satisfies the boundary conditions. Then substitute into the ODE to obtain $f$.
- Compute the numerical approximation with the given $f$, and use it to calculate the error.

For example, let us try
$$
u(x) = e^{\cos(\pi x)}.
$$
Then
$$
u'(x) = -\pi \sin(\pi x) e^{\cos(\pi x)}
$$
and
$$
f(x) = u''(x) = \pi^2 e^{\cos(\pi x)} \left( \sin^2(\pi x) - \cos(\pi x) \right).
$$
The corresponding boundary conditions are
$$
u(0) = e, u(1) = e^{-1}.
$$

### Setting up the matrix

In [ ]:
# Boundary condition values
al = exp(1)
be = exp(-1)

# Set grid resolution
m = 49
h = 1/(m+1)
x = np.linspace(h, 1-h, m)

# Generate the centered difference differentiation matrix for u''
f = 1/(h*h)
A = np.diag(-np.ones(m)*f*2)+np.diag(np.ones(m-1)*f, 1) + \
    np.diag(np.ones(m-1)*f, -1)

What does matrix $A$ look like? Let us visualize it.

In [ ]:
plt.spy(A)
plt.show()

It is indeed a sparse matrix with only entries along the diagonals.

### Solving for the matrix equation

In [ ]:
# Define the right-hand side vector
F = np.array([pi*pi*exp(cos(pi*z))*(sin(pi*z)**2-cos(pi*z)) for z in x])

# Modify the first and last entries of F to handle the Dirichlet conditions
F[0] -= al*f
F[m-1] -= be*f

# Solve the linear system
U = np.linalg.solve(A, F)

### Computing the error

In [ ]:
# Store the results for later analysis
results = []

# Print the exact and approximate solutions, and compute the infinity norm error
infnorm = 0
for i in range(m):
    uex = exp(cos(pi*x[i]))
    E = U[i]-uex
    if abs(E) > infnorm:
        infnorm = abs(E)
    print(x[i], U[i], uex, E)
    results.append((x[i], U[i], uex, E))
print("# Infinity norm error:", infnorm)

# Extract x, U, uex, E from results for plotting
x_vals, U_vals, uex_vals, E_vals = zip(*results)

### Plotting the results

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 4), sharex=True, dpi=300)

# Plot the numerical and exact solutions
ax[0].plot(x_vals, U_vals, color='tab:blue', label='Numerical', marker='o')
ax[0].plot(x_vals, uex_vals, color='tab:orange', label='Exact', marker='x')

# Plot the absolute error
ax[1].plot(x_vals, np.abs(E_vals), color='tab:green', label='Pointwise absolute error', marker='s')
ax[1].plot(x_vals, [infnorm]*len(x_vals), 'r--', label='Infinity norm error')

# Formatting
ax[0].set_ylabel('$u(x)$')
ax[0].legend(loc='best')
ax[1].set_xlabel('$x$')
ax[1].legend(loc='best')
ax[1].set_ylim(-0.0001, 0.0008)

plt.show()

## 1D Newton's method

### Implement the 1D Newton's method for root-finding

In [ ]:
# Function to perform root-finding on
def f(x):
    return sin(x)-0.9+x**3

# Analytical derivative
def df(x):
    return cos(x)+3*x*x


# Choose starting point for Newton iteration
xa = 1

# Store the results for later analysis
results = [xa]

for k in range(20):

    # Print out the current iterate, and the function value there
    print("%17.10g %17.10g" % (xa, f(xa)))

    # Perform Newton step
    xa = xa-f(xa)/df(xa)

    # Store the current iterate
    results.append(xa)

### Visualize

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=300)

# Plot the function to root find
xx = np.linspace(-1, 2, 400)
ax.plot(xx, np.sin(xx) - 0.9 + xx**3, lw=2, label=r'$f(x) = \sin(x)-0.9+x^3$')

# Plot the Newton iterates
# Plot Newton iterates
ax.plot(results, [0]*len(results), 'rx', label='Newton iterates')
ax.plot(results, [sin(x)-0.9+x**3 for x in results], 'rx', label='Newton iterates', alpha=0.5)
for x_iter, y_iter in zip(results, [sin(xi)-0.9+xi**3 for xi in results]):
    ax.plot([x_iter, x_iter], [0, y_iter], 'r--', lw=1)

# Plot tangent lines at each Newton iterate
for x0 in results:
    y0 = sin(x0) - 0.9 + x0**3
    slope = cos(x0) + 3*x0**2
    # Tangent line: y = y0 + slope*(x - x0)
    x_tan = np.linspace(x0 - 0.5, x0 + 0.5, 20)
    y_tan = y0 + slope * (x_tan - x0)
    ax.plot(x_tan, y_tan, 'g--', alpha=0.5)

# Formatting
ax.axhline(0, color='gray', linestyle='--')
ax.set_xlabel('$x$')
ax.set_ylabel('$f(x)$')
ax.set_xlim(0., 1.5)
ax.set_ylim(-2, 2)
ax.legend()
ax.grid(True)

plt.show()

## Gaussian quadrature

Gaussian quadrature is a method of numerically evaluating integrals exactly. The $n$-point Gaussian quadrature scheme samples an integrand at $\{x_1, x_2, \ldots, x_n\}$ and sums their weights $\{w_1, w_2, \ldots, w_n\}$ to obtain the approximation
$$
\int_{-1}^1 f(x) \, dx \approx \sum_{k=1}^n w_k f(x_k)
$$
Remarkably, **$n$ point Gaussian quadrature can integrate polynomials up to degree $2n-1$ exactly!** Further, it gives accurate results across a wide range of integrands.

The $n$-point Gaussian quadrature scheme can be derived analytically using orthogonal polynomials (see [Harvard AM205 Video 3.4](https://youtu.be/wNkXmXyGHo4?si=BXPmPTKloxgfgSK9)). As a nonlinear root-finding example, we aim to derive the two-point Gaussian quadrature points $\{x_1, x_2\}$ and weights $\{w_1, w_2\}
$.

We require
$$
\begin{aligned}
2 &= \int_{-1}^1 1 \, dx &= w_1 + w_2, \\
0 &= \int_{-1}^1 x \, dx &= w_1 x_1 + w_2 x_2, \\
\frac{2}{3} &= \int_{-1}^1 x^2 \, dx &= w_1 x_1^2 + w_2 x_2^2, \\
0 &= \int_{-1}^1 x^3 \, dx &= w_1 x_1^3 + w_2 x_2^3.
\end{aligned}
$$
We search for a root of
$$
G(x_1, x_2, w_1, w_2) = \begin{pmatrix}
w_1 + w_2 - 2 \\
w_1 x_1 + w_2 x_2 \\
w_1 x_1^2 + w_2 x_2^2 - 2/3 \\
w_1 x_1^3 + w_2 x_2^3
\end{pmatrix}.
$$

See the code below, which shows a custom implementation of the Newton method for this problem, and demonstrates Python’s `fsolve`. The analytical solution is
$$
x_1 = -\frac{1}{\sqrt{3}}, x_2 = \frac{1}{\sqrt{3}}, w_1 = 1, w_2 = 1.
$$

### Define the functions

In [ ]:
# Function to root-find on
def f(x):
    print(x)
    (x1, x2, w1, w2) = (x[0], x[1], x[2], x[3])
    x1s = x1*x1
    x2s = x2*x2
    f = np.array([w1+w2-2, w1*x1+w2*x2,
                  w1*x1s+w2*x2s-2./3, w1*x1s*x1+w2*x2s*x2])
    return f

# Analytical Jacobian
def fprime(x):
    print("# fprime called")
    (x1, x2, w1, w2) = (x[0], x[1], x[2], x[3])
    x1s = x1*x1
    x2s = x2*x2
    return np.array([[0, 0, 1, 1], 
                     [w1, w2, x1, x2],
                     [2*w1*x1, 2*w2*x2, x1s, x2s], 
                     [3*w1*x1s, 3*w2*x2s, x1s*x1, x2s*x2]])

### Newton's method in three modes

In [ ]:
# Mode: 0 = custom Newton method
#       1 = fsolve with Jacobian
#       2 = fsolve without Jacobian
mode = 2

# Initial condition
x0 = np.array([-1., 1., 2., 2.])

# Custom Newton method implementation
if mode == 0:
    xk = x0
    fk = f(xk)
    err = np.linalg.norm(fk)
    while err > 1e-14:
        xk -= np.linalg.solve(fprime(xk), fk)
        fk = f(xk)
        err = np.linalg.norm(fk)

# Call fsolve routines with and without Jacobian
elif mode == 1:
    xk = fsolve(f, x0, fprime=fprime)
else:
    xk = fsolve(f, x0)

# Print solution
print("\n# Solution is", xk)